In [0]:
# Notebook: 03_model_inference.py

import mlflow
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

# --- Configuração do Modelo Registrado ---
MODEL_NAME = "Insurance_Cost_LR_Model"
MODEL_ALIAS = "Production" 
MODEL_URI = f"models:/{MODEL_NAME}@{MODEL_ALIAS}" 

# NOVO: DEFINIÇÃO DO LOCAL TEMPORÁRIO NO VOLUME (SOLUÇÃO DO ERRO)
UC_VOLUME_TMP_PATH = "/Volumes/dev_catalogue/staging_schema/raw_data_volume/mlflow_temp"

print(f"URI de carregamento: {MODEL_URI}") 


# 1. Simulação de Novos Dados de Entrada
schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("sex", StringType(), True),
    StructField("bmi", DoubleType(), True),
    StructField("children", IntegerType(), True),
    StructField("smoker", StringType(), True),
    StructField("region", StringType(), True)
])

new_data = [
    (28, "female", 33.0, 1, "no", "northwest"),
    (55, "male", 30.5, 2, "yes", "southeast"),
    (19, "female", 25.0, 0, "no", "southwest")
]

df_new_customers = spark.createDataFrame(new_data, schema)

print("Novos Dados de Entrada (Clientes para Previsão):")
df_new_customers.show()


# 2. Carregamento do Modelo e Transformação
model = mlflow.spark.load_model(MODEL_URI, dfs_tmpdir=UC_VOLUME_TMP_PATH)

print(f"✅ Modelo carregado com sucesso do MLflow Registry: {MODEL_URI}")

# 3. Execução da Inferência
# O modelo carregado (PipelineModel) aplica a StringIndexer, OneHotEncoder, VectorAssembler, e a Previsão
# Isso cria a coluna 'features' e, em seguida, a coluna 'prediction'.
df_predictions = model.transform(df_new_customers) # <--- Esta linha agora deve criar a coluna 'features'

# 3. Execução da Inferência
df_predictions = model.transform(df_new_customers)


# 4. Exibição e Limpeza dos Resultados
df_final_results = df_predictions.select(
    "age", 
    "smoker", 
    "region", 
    F.col("prediction").alias("estimated_insurance_cost")
)

print("\nResultados da Inferência (Custo Estimado do Seguro):")
df_final_results.show()

In [0]:
# --- Configuração da Tabela GOLD ---
GOLD_TABLE = "dev_catalogue.staging_schema.gold_insurance_predictions"

# 5. Salvar na Camada GOLD
df_final_results.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(GOLD_TABLE)

print(f"\n✅ Camada GOLD (Inferência) salva com sucesso em: {GOLD_TABLE}")